## Dependecies

In [1]:
# Dependencies and Setup
import pandas as pd
from sqlalchemy import create_engine

 ## Store CSV into DataFrame

In [2]:
sniiv_file = "Resources/sniiv.csv"
sniiv_df = pd.read_csv(sniiv_file, encoding='latin-1')
sniiv_df.head()

,Año,Mes acumulado,Clave Entidad INEGI,Entidad Federativa,Clave Municipio INEGI,Municipio,Modalidad,Genero,Rango de Edad,Rango de Ingresos UMA,Valor de la Vivienda,Acciones,Monto
0,2020,12,15,México,31,Chimalhuacán,Mejoramientos,Hombre,60 o más,2.61 a 4.00,Económica,7,"287,400.24"
1,2020,12,15,México,31,Chimalhuacán,Mejoramientos,Hombre,30 a 59,2.61 a 4.00,Económica,23,"930,547.12"
2,2020,12,15,México,31,Chimalhuacán,Mejoramientos,Hombre,29 o menos,2.61 a 4.00,Económica,10,"341,732.00"
3,2020,12,15,México,31,Chimalhuacán,Mejoramientos,Hombre,60 o más,2.6 o menos,Económica,3,"126,613.52"
4,2020,12,15,México,31,Chimalhuacán,Mejoramientos,Hombre,30 a 59,2.6 o menos,Económica,9,"343,699.68"


## Data Transforming

In [3]:
# Get a list of all of our columns for easy reference.

sniiv_df.columns

Index(['Año', 'Mes acumulado', 'Clave Entidad INEGI', 'Entidad Federativa',
       'Clave Municipio INEGI', 'Municipio', 'Modalidad', 'Genero',
       'Rango de Edad', 'Rango de Ingresos UMA', 'Valor de la Vivienda',
       'Acciones', 'Monto'],
      dtype='object')

In [4]:
# Reduce DF to allocate just the needed columns (indicatorid / ageid / estimateid / datasetid / raceid / yearid)

sniiv_analysis_df = sniiv_df.loc[:, ["Clave Entidad INEGI", "Entidad Federativa", "Clave Municipio INEGI", 
                                     "Municipio", "Genero", "Rango de Edad","Monto"]]
sniiv_analysis_df

,Clave Entidad INEGI,Entidad Federativa,Clave Municipio INEGI,Municipio,Genero,Rango de Edad,Monto
0,15,México,31,Chimalhuacán,Hombre,60 o más,"287,400.24"
1,15,México,31,Chimalhuacán,Hombre,30 a 59,"930,547.12"
2,15,México,31,Chimalhuacán,Hombre,29 o menos,"341,732.00"
3,15,México,31,Chimalhuacán,Hombre,60 o más,"126,613.52"
4,15,México,31,Chimalhuacán,Hombre,30 a 59,"343,699.68"
...,...,...,...,...,...,...,...
4931,12,Guerrero,1,Acapulco de Juárez,Hombre,60 o más,"660,000.00"
4932,21,Puebla,81,Ixcamilpa de Guerrero,Hombre,30 a 59,"160,000.00"
4933,26,Sonora,29,Guaymas,Hombre,30 a 59,"180,000.00"
4934,26,Sonora,29,Guaymas,Hombre,60 o más,"200,000.00"


In [5]:
# Check what type are each variable.

sniiv_analysis_df.dtypes

Clave Entidad INEGI       int64
Entidad Federativa       object
Clave Municipio INEGI     int64
Municipio                object
Genero                   object
Rango de Edad            object
Monto                    object
dtype: object

In [6]:
# Convert Claves to string to be able to concatenate

sniiv_analysis_df["Clave Entidad INEGI"] = sniiv_analysis_df["Clave Entidad INEGI"].astype(str)
sniiv_analysis_df["Clave Municipio INEGI"] = sniiv_analysis_df["Clave Municipio INEGI"].astype(str)

sniiv_analysis_df

,Clave Entidad INEGI,Entidad Federativa,Clave Municipio INEGI,Municipio,Genero,Rango de Edad,Monto
0,15,México,31,Chimalhuacán,Hombre,60 o más,"287,400.24"
1,15,México,31,Chimalhuacán,Hombre,30 a 59,"930,547.12"
2,15,México,31,Chimalhuacán,Hombre,29 o menos,"341,732.00"
3,15,México,31,Chimalhuacán,Hombre,60 o más,"126,613.52"
4,15,México,31,Chimalhuacán,Hombre,30 a 59,"343,699.68"
...,...,...,...,...,...,...,...
4931,12,Guerrero,1,Acapulco de Juárez,Hombre,60 o más,"660,000.00"
4932,21,Puebla,81,Ixcamilpa de Guerrero,Hombre,30 a 59,"160,000.00"
4933,26,Sonora,29,Guaymas,Hombre,30 a 59,"180,000.00"
4934,26,Sonora,29,Guaymas,Hombre,60 o más,"200,000.00"


In [7]:
# Check what type are each variable.

sniiv_analysis_df.dtypes

Clave Entidad INEGI      object
Entidad Federativa       object
Clave Municipio INEGI    object
Municipio                object
Genero                   object
Rango de Edad            object
Monto                    object
dtype: object

In [8]:
# Making a coy of Clave Entidad INEGI  & Clave Municipio INEGI to be able to concatenate

new = sniiv_analysis_df["Clave Municipio INEGI"].copy()

In [9]:
# Concatenating Clave Entidad INEGI  & Clave Municipio INEGI with name column ID

sniiv_analysis_df["Clave Entidad INEGI"] = sniiv_analysis_df["Clave Entidad INEGI"].str.cat(new,sep="-")

sniiv_analysis_df

,Clave Entidad INEGI,Entidad Federativa,Clave Municipio INEGI,Municipio,Genero,Rango de Edad,Monto
0,15-31,México,31,Chimalhuacán,Hombre,60 o más,"287,400.24"
1,15-31,México,31,Chimalhuacán,Hombre,30 a 59,"930,547.12"
2,15-31,México,31,Chimalhuacán,Hombre,29 o menos,"341,732.00"
3,15-31,México,31,Chimalhuacán,Hombre,60 o más,"126,613.52"
4,15-31,México,31,Chimalhuacán,Hombre,30 a 59,"343,699.68"
...,...,...,...,...,...,...,...
4931,12-1,Guerrero,1,Acapulco de Juárez,Hombre,60 o más,"660,000.00"
4932,21-81,Puebla,81,Ixcamilpa de Guerrero,Hombre,30 a 59,"160,000.00"
4933,26-29,Sonora,29,Guaymas,Hombre,30 a 59,"180,000.00"
4934,26-29,Sonora,29,Guaymas,Hombre,60 o más,"200,000.00"


In [10]:
# Rename the columns a create new data frame / Reset the index

sniiv_analysis_adjusted_df = sniiv_analysis_df.rename(columns = {"Clave Entidad INEGI":"ID", "Entidad Federativa":"Entidad_Fed",
                                   "Municipio":"Municipio", "Genero":"Genero", "Rango de Edad":"Rango_Edad",
                                   "Monto":"Prestamo"}, inplace=False)

sniiv_analysis_adjusted_df = sniiv_analysis_adjusted_df.reset_index(drop=True)

sniiv_analysis_adjusted_df

,ID,Entidad_Fed,Clave Municipio INEGI,Municipio,Genero,Rango_Edad,Prestamo
0,15-31,México,31,Chimalhuacán,Hombre,60 o más,"287,400.24"
1,15-31,México,31,Chimalhuacán,Hombre,30 a 59,"930,547.12"
2,15-31,México,31,Chimalhuacán,Hombre,29 o menos,"341,732.00"
3,15-31,México,31,Chimalhuacán,Hombre,60 o más,"126,613.52"
4,15-31,México,31,Chimalhuacán,Hombre,30 a 59,"343,699.68"
...,...,...,...,...,...,...,...
4931,12-1,Guerrero,1,Acapulco de Juárez,Hombre,60 o más,"660,000.00"
4932,21-81,Puebla,81,Ixcamilpa de Guerrero,Hombre,30 a 59,"160,000.00"
4933,26-29,Sonora,29,Guaymas,Hombre,30 a 59,"180,000.00"
4934,26-29,Sonora,29,Guaymas,Hombre,60 o más,"200,000.00"


## Cleaned SNIIV Table

In [11]:
# Select the final columns for the 

sniiv_final_df = sniiv_analysis_adjusted_df[["ID","Entidad_Fed","Municipio", "Genero", "Rango_Edad", "Prestamo"]].copy()
sniiv_final_df

,ID,Entidad_Fed,Municipio,Genero,Rango_Edad,Prestamo
0,15-31,México,Chimalhuacán,Hombre,60 o más,"287,400.24"
1,15-31,México,Chimalhuacán,Hombre,30 a 59,"930,547.12"
2,15-31,México,Chimalhuacán,Hombre,29 o menos,"341,732.00"
3,15-31,México,Chimalhuacán,Hombre,60 o más,"126,613.52"
4,15-31,México,Chimalhuacán,Hombre,30 a 59,"343,699.68"
...,...,...,...,...,...,...
4931,12-1,Guerrero,Acapulco de Juárez,Hombre,60 o más,"660,000.00"
4932,21-81,Puebla,Ixcamilpa de Guerrero,Hombre,30 a 59,"160,000.00"
4933,26-29,Sonora,Guaymas,Hombre,30 a 59,"180,000.00"
4934,26-29,Sonora,Guaymas,Hombre,60 o más,"200,000.00"
